# Generative Flow Network Demo
From https://colab.research.google.com/drive/1fUMwgu2OhYpQagpzU5mhe9_Esib3Q2VR


In [ ]:
import os
import sys

import matplotlib.cm as cm
import matplotlib.pyplot as matplotlib_pyplot
import numpy as numpy
import torch
import tqdm
from torch.distributions.categorical import Categorical

# Add the project root directory to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

import gflownet as gflownet

In [ ]:
def plot_faces(faces):
    f, ax = matplotlib_pyplot.subplots(1, len(faces))
    for i in range(len(faces)):
        if len(faces) > 1:
            matplotlib_pyplot.sca(ax[i])
        gflownet.Face.draw_face(faces[i])

smiling_face = gflownet.Face(['smile', 'left_eb_down', 'right_eb_down'])
frowning_face = gflownet.Face(['frown', 'left_eb_up', 'right_eb_up'])
plot_faces([gflownet.Face(['left_eb_up', 'right_eb_up']), gflownet.Face(['frown']), gflownet.Face(['smile'])])
plot_faces([gflownet.Face(['left_eb_up', 'left_eb_down']), gflownet.Face(['right_eb_up', 'right_eb_down']),
            gflownet.Face(['left_eb_up', 'left_eb_down', 'right_eb_up', 'right_eb_down']),
            gflownet.Face(['smile', 'frown'])])
plot_faces([frowning_face, smiling_face])

In [ ]:
enumerated_states, transitions = gflownet.Face.enumerate_states_transitions(gflownet.Face.sorted_keys)
unique_states = []
for face in enumerated_states:
    if set(face.patches) not in [set(u.patches) for u in unique_states]:
        unique_states.append(face)

In [ ]:
def face_hash(face):
    return tuple([i in face for i in gflownet.Face.sorted_keys])

In [ ]:
lens = [len([i for i in unique_states if len(i) == j]) for j in range(4)]
levels = [sorted([i for i in unique_states if len(i) == j]) for j in range(4)]
f = matplotlib_pyplot.figure(figsize=(8, 8))
face2pos = {}
for i, (level, L) in enumerate(zip(levels, lens)):
    for j, face in enumerate(level):
        ax = f.add_axes([j / L, i / 4, 1 / L, 1 / 6])
        face.draw_face()
        face2pos[face_hash(face)] = (j / L + 0.5 / L, i / 4)
ax = f.add_axes([0, 0, 1, 1])
matplotlib_pyplot.sca(ax)
matplotlib_pyplot.gca().set_facecolor((0, 0, 0, 0))
matplotlib_pyplot.xlim(0, 1)
matplotlib_pyplot.ylim(0, 1)
for a, b in transitions[1:]:
    pa, pb = face2pos[face_hash(a)], face2pos[face_hash(b)]
    if not len(b): continue
    lb = int(pb[1] * 4)
    la = int(pa[1] * 4)
    ws = [1 / 6, 1 / 6, 0.13, 0.11]
    matplotlib_pyplot.arrow(pa[0], pa[1] + ws[la], pb[0] - pa[0], pb[1] - pa[1] - ws[lb], head_width=0.01, width=0.003,
                            ec=(0., 0.5, 0), fc=(0, 0, 0),
                            length_includes_head=True)
    matplotlib_pyplot.axis('off')

In [ ]:
def face_parents(state):
    parent_states = []  # states that are parents of state
    parent_actions = []  # actions that lead from those parents to state
    for face_part in state:
        # For each face part, there is a parent without that part
        parent_states.append([i for i in state if i != face_part])
        # The action to get there is the corresponding index of that face part
        parent_actions.append(gflownet.Face.sorted_keys.index(face_part))
    return parent_states, parent_actions

# Flow Matching
https://arxiv.org/abs/2106.04399

In [ ]:
# Instantiate model and optimizer
F_sa = gflownet.FlowModel(512)
opt = torch.optim.Adam(F_sa.parameters(), 3e-4)

# Let's keep track of the losses and the faces we sample
losses = []
sampled_faces = []
# To not complicate the code, I'll just accumulate losses here and take a
# gradient step every `update_freq` episode.
minibatch_loss = 0
update_freq = 4
for episode in tqdm.tqdm(range(50000), ncols=40):
    # Each episode starts with an "empty state"
    state = []
    # Predict F(s, a)
    edge_flow_prediction = F_sa(gflownet.Face(state).face_to_tensor())
    for t in range(3):
        # The policy is just normalizing, and gives us the probability of each action
        policy = edge_flow_prediction / edge_flow_prediction.sum()
        # Sample the action
        action = Categorical(probs=policy).sample()
        # "Go" to the next state
        new_state = state + [gflownet.Face.sorted_keys[action]]

        # Now we want to compute the loss, we'll first enumerate the parents
        parent_states, parent_actions = face_parents(new_state)
        # And compute the edge flows F(s, a) of each parent
        px = torch.stack([gflownet.Face(p).face_to_tensor() for p in parent_states])
        pa = torch.tensor(parent_actions).long()
        parent_edge_flow_preds = F_sa(px)[torch.arange(len(parent_states)), pa]
        # Now we need to compute the reward and F(s, a) of the current state,
        # which is currently `new_state`
        if t == 2:
            # If we've built a complete face, we're done, so the reward is > 0
            # (unless the face is invalid)
            reward = gflownet.Face(new_state).face_reward()
            # and since there are no children to this state F(s,a) = 0 \forall a
            edge_flow_prediction = torch.zeros(6)
        else:
            # Otherwise we keep going, and compute F(s, a)
            reward = 0
            edge_flow_prediction = F_sa(gflownet.Face(new_state).face_to_tensor())

        # The loss as per the equation above
        flow_mismatch = (parent_edge_flow_preds.sum() - edge_flow_prediction.sum() - reward).pow(2)
        minibatch_loss += flow_mismatch  # Accumulate
        # Continue iterating
        state = new_state

    # We're done with the episode, add the face to the list, and if we are at an
    # update episode, take a gradient step.
    sampled_faces.append(gflownet.Face(state))
    if episode % update_freq == 0:
        losses.append(minibatch_loss.item())
        minibatch_loss.backward()
        opt.step()
        opt.zero_grad()
        minibatch_loss = 0

In [ ]:
matplotlib_pyplot.figure(figsize=(10, 3))
matplotlib_pyplot.plot(losses)
matplotlib_pyplot.yscale('log')

In [ ]:
f, ax = matplotlib_pyplot.subplots(8, 8, figsize=(4, 4))
print('Ratio of faces with a smile:', sum(['smile' in i.patches for i in sampled_faces[-128:]]) / 128)
print('Ratio of valid faces:', sum([i.face_reward() > 0 for i in sampled_faces[-128:]]) / 128)
for i, face in enumerate(sampled_faces[-64:]):
    matplotlib_pyplot.sca(ax[i // 8, i % 8])
    face.draw_face()

In [ ]:
lens = [len([i for i in unique_states if len(i) == j]) for j in range(4)]
levels = [sorted([i for i in unique_states if len(i) == j]) for j in range(4)]
f = matplotlib_pyplot.figure(figsize=(8, 8))
face2pos = {}
for i, (level, L) in enumerate(zip(levels, lens)):
    for j, face in enumerate(level):
        ax = f.add_axes([j / L, i / 4, 1 / L, 1 / 6])
        gflownet.Face(face).draw_face()
        face2pos[face_hash(face)] = (j / L + 0.5 / L, i / 4)
        Fstate = F_sa(torch.tensor(face_hash(face)).float())
        #print(face, Fstate.data)
ax = f.add_axes([0, 0, 1, 1])
matplotlib_pyplot.sca(ax)
matplotlib_pyplot.gca().set_facecolor((0, 0, 0, 0))
matplotlib_pyplot.xlim(0, 1)
matplotlib_pyplot.ylim(0, 1)
for a, b in transitions[1:]:
    if not len(b):
        continue
    pa, pb = face2pos[face_hash(a)], face2pos[face_hash(b)]
    lb = int(pb[1] * 4)
    Fstate = F_sa(torch.tensor(face_hash(a)).float())
    Fa = Fstate[gflownet.Face.sorted_keys.index([i for i in b if i not in a][0])].item()
    c = cm.brg(Fa / 3)
    lb = int(pb[1] * 4)
    la = int(pa[1] * 4)
    ws = [1 / 6, 1 / 6, 0.13, 0.11]
    matplotlib_pyplot.arrow(pa[0], pa[1] + ws[la], pb[0] - pa[0], pb[1] - pa[1] - ws[lb], head_width=0.01, width=0.003,
                            ec=c, fc=c,
                            length_includes_head=True)
    matplotlib_pyplot.axis('off')
ax = f.add_axes([1, 0.2, 0.05, 0.6])
matplotlib_pyplot.sca(ax)
f.colorbar(cm.ScalarMappable(norm=cm.colors.Normalize(vmin=0, vmax=3), cmap=cm.brg), cax=ax, label='Edge Flow');

In [ ]:
F_sa(gflownet.Face([]).face_to_tensor()).sum()

In [ ]:
# Instantiate model and optimizer
model = gflownet.TBModel(512)
opt = torch.optim.Adam(model.parameters(), 3e-4)

# Let's keep track of the losses and the faces we sample
tb_losses = []
tb_sampled_faces = []
# To not complicate the code, I'll just accumulate losses here and take a
# gradient step every `update_freq` episode.
minibatch_loss = 0
update_freq = 2

logZs = []
for episode in tqdm.tqdm(range(50000), ncols=40):
    # Each episode starts with an "empty state"
    state = []
    # Predict P_F, P_B
    P_F_s, P_B_s = model(gflownet.Face(state).face_to_tensor())
    total_P_F = 0
    total_P_B = 0
    for t in range(3):
        # Here P_F is logits, so we want the Categorical to compute the softmax for us
        cat = Categorical(logits=P_F_s)
        action = cat.sample()
        # "Go" to the next state
        new_state = state + [gflownet.Face.sorted_keys[action]]
        # Accumulate the P_F sum
        total_P_F += cat.log_prob(action)

        if t == 2:
            # If we've built a complete face, we're done, so the reward is > 0
            # (unless the face is invalid)
            reward = torch.tensor(gflownet.Face(new_state).face_reward()).float()
        # We recompute P_F and P_B for new_state
        P_F_s, P_B_s = model(gflownet.Face(new_state).face_to_tensor())
        # Here we accumulate P_B, going backwards from `new_state`. We're also just
        # going to use opposite semantics for the backward policy. I.e., for P_F action
        # `i` just added the face part `i`, for P_B we'll assume action `i` removes
        # face part `i`, this way we can just keep the same indices.
        total_P_B += Categorical(logits=P_B_s).log_prob(action)

        # Continue iterating
        state = new_state

    # We're done with the trajectory, let's compute its loss. Since the reward can
    # sometimes be zero, instead of log(0) we'll clip the log-reward to -20.
    loss = (model.logZ + total_P_F - torch.log(reward).clip(-20) - total_P_B).pow(2)
    minibatch_loss += loss

    # Add the face to the list, and if we are at an
    # update episode, take a gradient step.
    tb_sampled_faces.append(state)
    if episode % update_freq == 0:
        tb_losses.append(minibatch_loss.item())
        minibatch_loss.backward()
        opt.step()
        opt.zero_grad()
        minibatch_loss = 0
        logZs.append(model.logZ.item())

In [ ]:
f, ax = matplotlib_pyplot.subplots(2, 1, figsize=(10, 6))
matplotlib_pyplot.sca(ax[0])
matplotlib_pyplot.plot(tb_losses)
matplotlib_pyplot.yscale('log')
matplotlib_pyplot.ylabel('loss')
matplotlib_pyplot.sca(ax[1])
matplotlib_pyplot.plot(numpy.exp(logZs))
matplotlib_pyplot.ylabel('estimated Z');

In [ ]:
f, ax = matplotlib_pyplot.subplots(8, 8, figsize=(4, 4))
print('Ratio of faces with a smile:', sum(['smile' in i for i in tb_sampled_faces[-128:]]) / 128)
print('Ratio of valid faces:', sum([gflownet.Face(i).face_reward() > 0 for i in tb_sampled_faces[-128:]]) / 128)
for i, face in enumerate(tb_sampled_faces[-64:]):
    matplotlib_pyplot.sca(ax[i // 8, i % 8])
    gflownet.Face(face).draw_face()

In [ ]:
model.logZ.exp()